In [1]:
import warnings
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
from datetime import datetime, time, timedelta

df = pd.read_csv('../db/out/wta_s.csv')

from sklearn.preprocessing import StandardScaler

df_2023 = df[df['date'].str.contains(
    '2023', na=False) | df['date'].str.contains('2022', na=False)]

df.drop(df_2023.index, inplace=True)

non_numeric_columns = df_2023.select_dtypes(['object']).columns

df_numeric_only = df_2023.drop(non_numeric_columns, axis=1)

df_2023 = df_numeric_only

df_2023 = df_2023.drop(['match_id'], axis=1)



df_2023 = df_2023.fillna(df_2023.median())

y_2023 = pd.DataFrame(df_2023['y'])
x_2023 = df_2023.drop(['y'], axis=1)

scaler = StandardScaler()

df_2023 = scaler.fit_transform(x_2023)

non_numeric_columns = df.select_dtypes(['object']).columns

df_numeric_only = df.drop(non_numeric_columns, axis=1)

df = df_numeric_only

df = df.drop(['match_id'], axis=1)

df = df.fillna(df.median())

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Y = pd.DataFrame(df['y'])
df = df.drop(['y'], axis=1)
X = df

scaler = StandardScaler()
X = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.1, random_state=45, stratify=Y)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28870 entries, 0 to 28869
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   win_percentage_vs            28870 non-null  float64
 1   bp_factor_vs                 28870 non-null  float64
 2   first_won_serve_vs           28870 non-null  float64
 3   second_won_serve_vs          28870 non-null  float64
 4   ace_probability_vs           28870 non-null  float64
 5   double_fault_probability_vs  28870 non-null  float64
 6   aceDf_vs                     28870 non-null  float64
 7   rank                         28870 non-null  float64
 8   rank_points                  28870 non-null  float64
 9   elo                          28870 non-null  float64
 10  glicko                       28870 non-null  float64
 11  ht                           28870 non-null  float64
 12  games_played                 28870 non-null  int64  
 13  win_percentage  

In [2]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=45, max_iter=200)


result = model.fit(X_train, y_train)
y_pred = model.predict(X_test)


basic_report = classification_report(y_test, y_pred, output_dict=True)
print("Trening")
print(basic_report)
basic_pre = basic_report['weighted avg']['precision']



pred_val = model.predict(x_2023)
basic_report = classification_report(y_2023, pred_val, output_dict=True)
print("Walidacja")
print(basic_report)
basic_pre_val = basic_report['weighted avg']['precision']

auc_basic = metrics.roc_auc_score(y_test, model.predict_proba(X_test)[::, 1])
print("AUC: ", auc_basic)

auc_val = metrics.roc_auc_score(y_2023, model.predict_proba(x_2023)[::, 1])
print("AUC: ", auc_val)

d:\OneDrivePW\OneDrive - Politechnika Warszawska\Documents\PracaInżynierska\Software\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Trening
{'0': {'precision': 0.6438082556591211, 'recall': 0.6752793296089385, 'f1-score': 0.6591683708248125, 'support': 1432.0}, '1': {'precision': 0.6642599277978339, 'recall': 0.6323024054982818, 'f1-score': 0.6478873239436619, 'support': 1455.0}, 'accuracy': 0.6536196744024939, 'macro avg': {'precision': 0.6540340917284775, 'recall': 0.6537908675536102, 'f1-score': 0.6535278473842372, 'support': 2887.0}, 'weighted avg': {'precision': 0.6541155583823034, 'recall': 0.6536196744024939, 'f1-score': 0.6534829107582818, 'support': 2887.0}}
Walidacja
{'0': {'precision': 0.6174089068825911, 'recall': 0.6389664804469274, 'f1-score': 0.6280027453671928, 'support': 1432.0}, '1': {'precision': 0.6234522942461762, 'recall': 0.6015460295151089, 'f1-score': 0.6123032904148784, 'support': 1423.0}, 'accuracy': 0.6203152364273204, 'macro avg': {'precision': 0.6204306005643836, 'recall': 0.6202562549810182, 'f1-score': 0.6201530178910356, 'support': 2855.0}, 'weighted avg': {'precision': 0.6204210750

d:\OneDrivePW\OneDrive - Politechnika Warszawska\Documents\PracaInżynierska\Software\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\OneDrivePW\OneDrive - Politechnika Warszawska\Documents\PracaInżynierska\Software\.venv\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
d:\OneDrivePW\OneDrive - Politechnika Warszawska\Documents\PracaInżynierska\Software\.venv\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [1]:
# import tensorflow
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import Input
from keras.layers import Dropout
from keras.layers import Dense
from keras.callbacks import History
# import keras_tuner as kt
from pathlib import Path
from time import strftime
# from scikeras.wrappers import KerasClassifier

# def get_run_logdir(root_logdir="my_run_logs"):
#     return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")


# tensorbord_cb = keras.callbacks.TensorBoard(
#     get_run_logdir(), profile_batch=(100, 200))


# history = History()

# # use keras API
# tf.random.set_seed(49)


# def build_model(hp):
#     n_hidden = hp.Int('n_hidden', min_value=1, max_value=4, step=1)
#     n_neurons = hp.Int('n_neurons', min_value=10, max_value=65, step=5)
#     learning_rate = hp.Float("learning_rate", min_value=0.0001, max_value=0.01,
#                              sampling="log")
#     kernel_regularizer = hp.Choice('kernel_regularizer', ['l1', 'l2'])
#     kernel_initializer = hp.Choice(
#         'kernel_initializer', ['he_normal', 'glorot_normal', 'lecun_normal'])
#     activation = hp.Choice('activation', ['relu', 'selu', 'sigmoid', 'tanh'])
#     optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
#     if optimizer == 'adam':
#         optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
#     elif optimizer == 'sgd':
#         optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
#     else:
#         optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)

#     model = keras.Sequential()
#     model.add(Input(shape=X_train.shape[1]))
#     for _ in range(n_hidden):
#         model.add(Dense(n_neurons, activation=activation,
#                   kernel_regularizer=kernel_regularizer, kernel_initializer=kernel_initializer))
#     model.add(Dense(1, activation='sigmoid'))
#     model.compile(optimizer=optimizer,
#                   loss='binary_crossentropy', metrics=['accuracy'])

#     return model


# model = tf.keras.Sequential()


# random_search_tuner = kt.BayesianOptimization(
#     build_model,
#     objective='val_accuracy',
#     max_trials=25,
#     executions_per_trial=3,
#     directory='my_dir',
#     project_name='wta_ann_bayesian_optimization',
#     overwrite=True)

# random_search_tuner.search(X_train, y_train, epochs=15,
#                            validation_data=(X_test, y_test))


# top3_models = random_search_tuner.get_best_models(num_models=3)
# best_model = top3_models[0]

# best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
# best_trial.summary()


# best_trial.metrics.get_last_value("val_accuracy")

In [ ]:
from scikeras.wrappers import KerasClassifier


def create_model():
    model = Sequential()
    model.add(Input(shape=X_train.shape[1]))
    model.add(Dense(96, activation='selu',
          kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l1_l2(0.1, 0.2)))
#     model.add(Dropout(0.4))
#     model.add(Dense(96, activation='relu',
#           kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(0.05)))
#     model.add(Dropout(0.6))
#     model.add(Dense(96, activation='relu',
#           kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(0.05)))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    optim = keras.optimizers.SGD(learning_rate=0.001969)
    model.compile(optimizer=optim,
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model


model = create_model()
model.fit(X_train, y_train, epochs=100, batch_size=256, validation_data=(x_2023, y_2023))
test_loss, test_acc = model.evaluate(x_2023, y_2023)
# # model = KerasClassifier(model=create_model, epochs=100, batch_size=32, validation_)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)




# opti_report = classification_report(y_test, y_pred, output_dict=True)
# print("Trening")
# print(opti_report)
# opti_pre = opti_report['weighted avg']['precision']
# opti_f1 = opti_report['weighted avg']['f1-score']
# opti_auc = metrics.roc_auc_score(y_test, model.predict(X_test))



# pred_val = model.predict(x_2023)
# opti_val_report = classification_report(y_2023, pred_val, output_dict=True)
# print("Walidacja")
# print(opti_val_report)
# opti_pre_val = opti_val_report['weighted avg']['precision']
# opti_f1_val = opti_val_report['weighted avg']['f1-score']
# opti_auc_val = metrics.roc_auc_score(y_2023, model.predict(x_2023))



: 

In [5]:
# import json

# filename = 'results.json'

# final_results = {
#     "prediction_basic": basic_pre,
#     "v_prcision_basic": basic_pre_val,
#     "prediction_optimized": opti_pre,
#     "v_prcision_optimized": opti_pre_val,
#     "f1_score_opt": opti_f1,
#     "f1_score_opt_val": opti_f1_val,
#     "auc_opt": float(opti_auc),
#     "auc_opt_val": float(opti_auc_val),
# }


# with open(filename, 'r') as file:
#     data = json.load(file)


# data['Sztuczna sieć neuronowa'] = (final_results)


# with open('results.json', 'w') as file:
#     json.dump(data, file, indent=4)